3.6.1-Credit Card Fraud

Assignment:

**Using this credit card fraud dataset develop an algorithm to predict fraud. Prioritize correctly finding fraud rather than correctly labeling non-fraudulent transactions.**

In [1]:
#import libraries
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#read dataset
df = pd.read_csv('creditcard.csv')

In [3]:
#Look at the shape of the dataset
df.shape

(284807, 31)

In [4]:
# Explore the features available in dataframe
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [5]:
# Count the occurrences of fraud and not fraud and print them
fraud = df['Class'].value_counts()
print(fraud)

0    284315
1       492
Name: Class, dtype: int64


In [6]:
# Print the ratio of fraud cases
print(fraud / len(df.index))

0    0.998273
1    0.001727
Name: Class, dtype: float64


**It appears there is not a lot of fraud detected in this dataset. We will have to make some adjustments to the models to get the best outcome of predicting fraud.**

In [7]:
#Create features by dropping the target variable "Class" and check the shape.
X = df.drop("Class", axis=1)
X.shape

(284807, 30)

In [8]:
#Create Target variable as a dataframe and get check the shape.
y = pd.DataFrame(df.Class)
y.shape

(284807, 1)

In [43]:
# Create the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# Fit a logistic regression model to our data
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_train, y_train)

# Get predictions
pred = log_reg.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**I played around with the LogisticRegression parameters to balance the data and try to get the lowest number of false negatives. Initially when I didn't use the class_weight the ratio was too high. When I added the balanced class_weight my false negatives were reduced by 18.25%**

In [44]:
# Print the classifcation report and confusion matrix
print('Classification report:\n', classification_report(y_test, pred))

conf_mat = confusion_matrix(y_true=y_test, y_pred=pred)

print('Confusion matrix:\n', conf_mat)

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99     85307
           1       0.06      0.93      0.10       136

   micro avg       0.97      0.97      0.97     85443
   macro avg       0.53      0.95      0.55     85443
weighted avg       1.00      0.97      0.99     85443

Confusion matrix:
 [[83160  2147]
 [   10   126]]


**Trying another algotirhm. Random Forest should help in getting great outcomes with unbalanced data.**

In [47]:
# define Random Forest Classifier
rand_for = ensemble.RandomForestClassifier(class_weight='balanced', random_state=42)

In [48]:
#Fit the classifier to the training set
rand_for.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=None, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [49]:
#Get perdictions
rand_pred = rand_for.predict(X_test)

In [50]:
# Print the classifcation report and confusion matrix
print('Classification report:\n', classification_report(y_test, rand_pred))

conf_mat = confusion_matrix(y_true=y_test, y_pred=rand_pred)

print('Confusion matrix:\n', conf_mat)

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.96      0.75      0.84       136

   micro avg       1.00      1.00      1.00     85443
   macro avg       0.98      0.87      0.92     85443
weighted avg       1.00      1.00      1.00     85443

Confusion matrix:
 [[85303     4]
 [   34   102]]


**The Random Classifier 